# Transcript from Lecture, October 28, 2021

In [ ]:
import sys

########################################
# Change the string in the line below! #
########################################
sys.path.append("/Users/gilbert/Documents/CS111-2021-fall/Python") 

import os
import time
import math
import numpy as np
import numpy.linalg as npla
import scipy
from scipy import linalg as spla
import scipy.sparse
import scipy.sparse.linalg
from scipy import integrate
import networkx as nx
import json
import cs111

##########################################################
# If this import for matplotlib doesn't work, try saying #
#   conda install -c conda-forge ipympl                  #
# at a shell prompt on your computer                     #
##########################################################
import matplotlib
%matplotlib ipympl

import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d




np.set_printoptions(precision = 4)

# Floating-point arithmetic is a leaky abstraction of the real numbers

In [ ]:
a = 4/3
a

In [ ]:
b = a-1
b

In [ ]:
c = 3*b
c

In [ ]:
d = 1-c
d

# 64-bit 2s complement integers

<b> Base 16 is "hexadecimal", with digits 0,1,2,3,...,f, four bits per digit

In [ ]:
cs111.bits

<b> int64 represents integers from $-2^{63}$ to $2^{63}-1$ using 64 bits = 16 hexadecimal digits

In [ ]:
print(cs111.int64_to_hex(0))

In [ ]:
print(cs111.int64_to_hex(3))

In [ ]:
print(cs111.int64_to_hex(10))

In [ ]:
print(cs111.int64_to_hex(1024))

In [ ]:
print(cs111.int64_to_hex(-1))

In [ ]:
print(cs111.int64_to_hex(-2))

In [ ]:
# largest positive int64
print(cs111.int64_to_hex(2**63 - 1))

In [ ]:
# most negative int64
print(cs111.int64_to_hex(-2**63))

# IEEE Standard 64-bit floating-point

In [ ]:
cs111.print_float64(0)

In [ ]:
cs111.print_float64(1)

In [ ]:
cs111.print_float64(42)

In [ ]:
cs111.print_float64(-1/3)

<b> float64 has both +0 and -0, but they compare as equal

In [ ]:
cs111.print_float64(-0.0)

In [ ]:
-0.0 == 0.0

# Floating-point infinity and NaN (not-a-number)

In [ ]:
np.inf

In [ ]:
1/np.inf

In [ ]:
# This is a bad flaw in python; it should give inf!

1.0 / 0.0

In [ ]:
np.inf + 100

In [ ]:
-2 * np.inf

In [ ]:
0 * np.inf

In [ ]:
cs111.print_float64(np.inf)

In [ ]:
cs111.print_float64(-np.inf)

In [ ]:
cs111.print_float64(np.nan)

<b> Is infinity equal to infinity?

In [ ]:
np.inf == np.inf

<b> NaN is not equal to anything, including itself!

In [ ]:
np.nan == 0

In [ ]:
np.nan == np.nan

In [ ]:
np.isnan(np.nan)

In [ ]:
np.isnan(np.inf)

In [ ]:
np.isnan(3.14)

# Properties of floating-point arithmetic

<b> Numbers that get big

In [ ]:
x = 1.0
while x < 2*x:
    print('x:', x, '    2x:', 2*x)
    lastx = x
    x = 2*x
print('x:', x, '    2x:', 2*x)

In [ ]:
cs111.print_float64(lastx)

<b> Numbers that get little

In [ ]:
x = 1.0
while x > x/2:
    print('x:', x, '    x/2:', 2*x)
    lastx = x
    x = x/2
print('x:', x, '    x/2:', x/2)

In [ ]:
cs111.print_float64(lastx)

<b>Machine epsilon

In [ ]:
x = 1.0
while 1 + x > 1:
    print('x:', x, '    1 + x:', 1+x)
    x = x/2
print('x:', x, '    1 + x:', 1+x)

<b> *machine epsilon* is the smallest x such that 1 + x > 1

In [ ]:
cs111.print_float64(x)

# Cancellation when subtracting almost equal numbers

In [ ]:
1/3

In [ ]:
a = 1
a + 1/3 - a

In [ ]:
a = 100
a + 1/3 - a

In [ ]:
a = 1000000
a + 1/3 - a

In [ ]:
a = 10**10
a + 1/3 - a

In [ ]:
a = 10**15
a + 1/3 - a

In [ ]:
a = 10**16
a + 1/3 - a

# Catastrophic cancellation in action: computing $(x-1)^7$

In [ ]:
# A simple function of a real number x
def f1(x):
    return (x-1)**7

# Same function as f1, but multiplied out and written as a polynomial in x
def f2(x):
    return x**7 - 7*x**6 + 21*x**5 - 35*x**4 + 35*x**3 - 21*x**2 + 7*x - 1


In [ ]:
x = 1
print('x:', x, '; (x-1)**7:', f1(x), ';  polynomial:', f2(x))

In [ ]:
x = 1.1
print('x:', x, '; (x-1)**7:', f1(x), ';  polynomial:', f2(x))

In [ ]:
x = 1.01
print('x:', x, '; (x-1)**7:', f1(x), ';  polynomial:', f2(x))

In [ ]:
x = 1.001
print('x:', x, '; (x-1)**7:', f1(x), ';  polynomial:', f2(x))

In [ ]:
xvals = np.linspace(.99, 1.01, 101)

y1 = []
y2 = []
for x in xvals:
    y1.append(f1(x))
    y2.append(f2(x))
y1 = np.array(y1)
y2 = np.array(y2)

%matplotlib inline
plt.figure()
plt.plot(xvals, y2, label = 'polynomial')
plt.plot(xvals, y1, label = '(x-1)**7')
plt.xlabel('x')
plt.ylabel('y')
plt.title('two ways to compute (x-1)**7')
plt.legend()

plt.figure()
plt.semilogy(xvals, np.abs((y1-y2)/y1))
plt.xlabel('x')
plt.ylabel('error')
plt.title('relative error in polynomial f2(x)')


# Exploring the limits of float64

<b> The largest float64 has a mantissa of all ones, and the largest non-infinity "exp" 7fe. The second-largest float64 differs in the last bit of the mantissa. See the hex below.

In [ ]:
fmax = ((1 << 53) - 1) / 2**52 * 2**1023
print('Largest float64:')
cs111.print_float64(fmax)
print()

fnextmax = ((1 << 53) - 2) / 2**52 * 2**1023
print('Second largest float64:')
cs111.print_float64(fnextmax)
print()

print('Difference:')
cs111.print_float64(fmax - fnextmax)

<b> The smallest normalized float64 has a mantissa of 1, so its "frac" is all zero, and it has the smallest 
possible nonzero "exp". The next larger normalized float64 has a "frac" of 1.

<b> The distance between those two numbers is smaller than any normalized float64. But it doesn't get rounded to zero -- it is a "denormal" float64. In fact, it's the smallest denormal float64, and thus the smallest positive float64 of all. You can read about denormals in the NCM chapter or the Wikipedia article.

In [ ]:
fmin = 1 * 2**(-1022)
print('Smallest normalized float64:')
cs111.print_float64(fmin)
print()

fnextmin = (1 + 1/2**52) * 2**(-1022) 
print('Second smallest normalized float64:')
cs111.print_float64(fnextmin)
print()

print('Difference (a denormal float64):')
cs111.print_float64(fnextmin - fmin)